<a href="https://colab.research.google.com/github/alexandrufalk/Background-generator/blob/Layout-page/CenterNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50


#Define the Backbone

In [ ]:
def create_backbone(input_shape=(512, 512, 3)):
    # Use ResNet50 as backbone without top layers, outputting feature maps
    backbone = ResNet50(input_shape=input_shape, include_top=False, weights="imagenet")
    backbone_output = backbone.get_layer("conv4_block6_out").output  # Choose a layer
    model = tf.keras.Model(inputs=backbone.input, outputs=backbone_output)
    return model

#Detection Head
The detection head will predict:

-Heatmap: to identify object centers.

-Offsets: to adjust for downsampling effects.

-Sizes: to predict the width and height of objects.


In [1]:
def create_detection_head(backbone_output, num_classes):
    # Heatmap head
    heatmap = tf.keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu")(backbone_output)
    heatmap = tf.keras.layers.Conv2D(num_classes, kernel_size=1, padding="same", activation="sigmoid", name="heatmap")(heatmap)

    # Offset head
    offset = tf.keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu")(backbone_output)
    offset = tf.keras.layers.Conv2D(2, kernel_size=1, padding="same", name="offset")(offset)  # 2 channels for x and y offsets

    # Size head
    size = tf.keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu")(backbone_output)
    size = tf.keras.layers.Conv2D(2, kernel_size=1, padding="same", name="size")(size)  # 2 channels for width and height

    return heatmap, offset, size


#Build the Complete CenterNet Model

In [ ]:
def create_centernet(input_shape=(512, 512, 3), num_classes=80):
    inputs = tf.keras.Input(shape=input_shape)
    backbone_output = create_backbone(input_shape)(inputs)
    heatmap, offset, size = create_detection_head(backbone_output, num_classes)
    model = tf.keras.Model(inputs=inputs, outputs=[heatmap, offset, size])
    return model


#Compile the Model

In [ ]:
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        alpha_factor = tf.where(tf.equal(y_true, 1), alpha, 1 - alpha)
        focal_weight = tf.where(tf.equal(y_true, 1), 1 - y_pred, y_pred)
        focal_weight = alpha_factor * tf.pow(focal_weight, gamma)
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        return focal_weight * bce
    return focal_loss_fixed

def smooth_l1_loss(y_true, y_pred):
    return tf.keras.losses.Huber()(y_true, y_pred)

centernet = create_centernet()
centernet.compile(optimizer='adam',
                  loss={'heatmap': focal_loss(),
                        'offset': smooth_l1_loss,
                        'size': smooth_l1_loss})